# Lec 14. Transfer Learning - VGG16 with CIFAR10

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data

from torchvision import models

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.utils
import numpy as np
import os
import subprocess

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Load Data

https://pytorch.org/docs/stable/torchvision/models.html

https://github.com/pytorch/vision/tree/master/torchvision/models

* 미리 사용할 모델의 Input 파악
* Transfer Learning은 이미 학습된 모델을 이용한다.

In [4]:
# VGG Net - input : 224*224 RGB

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

test_loader = Data.DataLoader(cifar10_test, batch_size=1, shuffle=False, num_workers=0)

Files already downloaded and verified


In [5]:
cifar10_test.test_data.shape

(10000, 32, 32, 3)

## Load VGG Model

In [6]:
vgg_model = models.vgg16(pretrained=True).cuda()

## Extract Features

In [7]:
# ILSVRC가 1,000개의 class를 분류하는 것이기 때문에 각 데이터에 대해 1,000개의 feature 생성됨
vgg_model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = vgg_model(images)   
    
    print(labels.numpy(), " -> ", len(outputs.data.cpu().numpy()[0]))
    print(outputs.data.cpu().numpy()[0])
    
    # 여기서 추출한 feature를 다른 머신 러닝 모델의 input으로 사용 가능.

    break

[3]  ->  1000
[ 1.38038886e+00  2.60775185e+00 -2.66931820e+00 -1.29921877e+00
 -2.48284101e+00  3.73538184e+00 -5.08292913e+00 -3.24098396e+00
 -3.20054960e+00 -6.49116135e+00  8.89340997e-01 -6.03052998e+00
 -4.12438202e+00 -4.51912546e+00 -3.57712674e+00 -4.20014000e+00
 -2.12850690e+00 -4.60034657e+00 -5.67246962e+00 -4.16573715e+00
 -2.06371522e+00 -1.06671262e+00 -4.98877811e+00 -3.84857869e+00
 -4.87575817e+00 -1.96659553e+00  5.32273985e-02 -4.38013852e-01
  8.84589434e-01  2.83307552e+00 -3.24980593e+00 -1.68612862e+00
  1.16592908e+00 -2.91229010e-01  3.47979933e-01 -2.95157433e-01
  8.43616664e-01 -3.82154155e+00  2.72055387e+00 -1.82603538e+00
 -3.47758675e+00  4.76205438e-01 -1.00875652e+00  3.78134131e+00
 -2.44167113e+00 -1.74806282e-01 -4.62706423e+00  3.69848460e-01
 -2.76754808e+00 -4.45083380e+00 -5.31234217e+00  5.73113632e+00
  5.20288646e-01 -9.08522308e-01 -2.02486897e+00 -5.56432104e+00
 -2.24783111e+00 -4.80476046e+00 -4.62084818e+00  1.36454928e+00
 -1.5540503